In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.model_selection import KFold


from sklearn.metrics import f1_score, roc_auc_score

In [2]:
pd.options.display.max_columns = 100

In [3]:
vendors = pd.read_csv('vendors.csv')

test_customers = pd.read_csv('test_customers.csv')
train_customers = pd.read_csv('train_customers.csv')

test_locations = pd.read_csv('test_locations.csv')
train_locations = pd.read_csv('train_locations.csv')

ss = pd.read_csv('SampleSubmission.csv')

orders = pd.read_csv('orders.csv')

C:\p\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
vendors.head()

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_akeed_delivering,discount_percentage,status,verified,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,Yes,0.0,1.0,1,11,EN,4.7,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,"{""primary_tags"":""7""}",1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,Yes,0.0,1.0,1,1,EN,4.5,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,"{""primary_tags"":""71""}",1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,Yes,0.0,1.0,1,11,EN,4.5,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,"{""primary_tags"":""46""}",1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,"{""primary_tags"":""32""}",1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


In [5]:
ss.head()

,CID X LOC_NUM X VENDOR,target
0,Z59FTQD X 0 X 243,0
1,0JP29SK X 0 X 243,0
2,0JP29SK X 1 X 243,0
3,0JP29SK X 2 X 243,0
4,0JP29SK X 3 X 243,0


In [6]:
ss['CID'] = ss['CID X LOC_NUM X VENDOR'].apply(lambda x: x.split(' X ')[0])
ss['LOC_NUM'] = ss['CID X LOC_NUM X VENDOR'].apply(lambda x: x.split(' X ')[1])
ss['VENDOR'] = ss['CID X LOC_NUM X VENDOR'].apply(lambda x: int(x.split(' X ')[2]))

In [7]:
ss['VENDOR'] .nunique()

100

In [8]:
ss['CID'].nunique()

10000

In [9]:
ss.shape

(1672000, 5)

In [10]:
train_customers.head()

,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [11]:
# gender  dob status verified  language created_at updated_at

In [12]:
train_customers['akeed_customer_id'].nunique()

34523

In [13]:
test_customers.head()

,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36
1,FWNUI71,Male,NaN,1,1,EN,2018-03-22 20:11:47,2018-03-22 20:11:47
2,LRX7BCH,Male,NaN,1,1,EN,2018-04-17 20:01:15,2018-04-17 20:01:15
3,D96DHMD,Male,NaN,1,1,EN,2018-04-29 22:35:52,2018-04-29 22:35:52
4,88Q8Y5V,Male,1997.0,1,1,EN,2018-05-05 12:38:09,2018-05-05 12:38:09


In [14]:
test_customers['akeed_customer_id'].nunique()

9753

In [15]:
train_locations.head()

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464


In [16]:
train_locations['customer_id'].nunique()

35400

In [17]:
test_locations.head()

,customer_id,location_number,location_type,latitude,longitude
0,Z59FTQD,0,NaN,126.032278,-9.106019
1,0JP29SK,0,Home,0.278709,-78.623847
2,0JP29SK,1,Home,0.124485,-78.605621
3,0JP29SK,2,NaN,-0.113891,-78.577449
4,0JP29SK,3,NaN,-0.848796,0.136726


In [18]:
len(test_locations)

16720

In [19]:
test_locations['customer_id'].nunique()

10000

In [20]:
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,2019-08-01 04:03:00,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [21]:
orders['akeed_order_id'] = orders['akeed_order_id'].fillna(-1).astype(int)

In [22]:
orders.groupby(['customer_id', 'LOCATION_NUMBER', 'vendor_id'])['item_count'].count().reset_index()

,customer_id,LOCATION_NUMBER,vendor_id,item_count
0,000THBA,0,148,0
1,002510Y,0,157,1
2,005ECL6,0,237,2
3,0075AM7,0,145,1
4,009UFS1,0,83,1
...,...,...,...,...
80137,ZZY3N0D,1,356,1
80138,ZZY3N0D,1,419,1
80139,ZZY3N0D,1,459,1
80140,ZZY3N0D,1,537,1


In [23]:
# customer_id, location_number   <- orders
train_locations

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464
...,...,...,...,...,...
59498,9PP42SA,2,NaN,-0.788515,-78.497721
59499,9PP42SA,3,Home,-1.445114,0.072558
59500,9PP42SA,4,NaN,-0.001785,0.431695
59501,HWELAU8,0,NaN,-0.066291,-78.583075


In [24]:
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,2019-08-01 04:03:00,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [25]:
# Make model for each vendor  

In [26]:
test_customers['akeed_customer_id'].value_counts().head(15)

JGRG3RQ    4
4X2Y51H    3
DSJRR66    3
P4MAWAE    3
DZORPPZ    2
WG3CZDG    2
YM0OIRI    2
JKC18K2    2
3DDZLWQ    2
II7YY1M    2
J8VMRWE    1
ZDGN2KP    1
V9PHR2L    1
FY82RQT    1
Y438XVE    1
Name: akeed_customer_id, dtype: int64

In [27]:
# вот тут надо удалить дубли их как раз 16
test_customers[test_customers['akeed_customer_id']=='II7YY1M']

,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
6820,II7YY1M,Female,1991.0,1,1,NaN,2019-11-25 20:17:56,2020-02-01 19:50:35
6821,II7YY1M,male,NaN,1,0,NaN,2019-11-25 20:17:58,2019-11-25 20:17:58


In [28]:
test_customers = test_customers.drop([6726,6855,6734,6718,6821 ] , axis=0)

In [29]:
test_customers = test_customers.drop([6668] , axis=0)

In [30]:
test_customers = test_customers.drop([6730,6731] , axis=0)

In [31]:
test_customers = test_customers.drop([6832,6831] , axis=0)

In [32]:
test_customers = test_customers.drop([6720,6721] , axis=0)

In [33]:
test_customers = test_customers.drop([6890, 6891, 6892] , axis=0)

In [34]:
def get_orders_by_vendor(orders, vendor_id):
    x=  orders.loc[orders['vendor_id'] == vendor_id, :].groupby(['customer_id',
                                                                    'LOCATION_NUMBER'])['akeed_order_id'].count().reset_index()

    x.columns = ['customer_id', 'location_number', 'akeed_order_id_count' ]
    return x

In [35]:
def get_lat_long_for_vendor(vendor_id):
    latitude_for_vendor_id = float(vendors.loc[vendors.id == vendor_id, 'latitude'])
    longitude_for_vendor_id = float(vendors.loc[vendors.id == vendor_id, 'longitude'])
    return latitude_for_vendor_id, longitude_for_vendor_id

In [36]:
def dist_from_customer_to_vendor( latitude_series, longitude_series, 
                                 latitude_for_vendor_id, longitude_for_vendor_id   ):
    return np.sqrt((latitude_series - latitude_for_vendor_id)**2 + (longitude_series - longitude_for_vendor_id)**2)

In [37]:
def language_process(language_series):
    return language_series.map( {'EN':1, 'np.nan':0 })

def gender_process(gender_series):
    gender_dict = { np.nan: 1 , 'Female':0, 'Male': 1, 'male': 1, 'Male  ': 1, '?????': 1, 'Female  ':0,
       'Female ':0, 'Female   ':0, 'Female    ':0, '  ':1}
    
    return gender_series.map(gender_dict)


def dob_process (dob_series):
    dob_series = dob_series.fillna(0)
    dob_series = dob_series.astype(int)
    return dob_series

In [38]:
def select_best_f1(y_true,y_score):
    best_threshold = 0
    best_f1_score = 0
    for threshold in np.linspace(0,1, 100):
        f1_score_test =f1_score(y_true= y_true, y_pred= (y_score>threshold)*1)
        if f1_score_test> best_f1_score:
            best_f1_score = f1_score_test
            best_threshold = threshold
    return best_f1_score, best_threshold

In [39]:
def make_solution_for_vendor(vendor_id):
    print("Make prediction for vendor {}".format(str(vendor_id)))
    orders_by_vendor = get_orders_by_vendor(orders, vendor_id)
    
    #make train subset for vendor_id with target
    train_set_for_vendor = pd.merge(train_locations,orders_by_vendor, how='left', on = ['customer_id',
                                                                                    'location_number'])
    
    train_set_for_vendor['akeed_order_id_count']= train_set_for_vendor['akeed_order_id_count'].fillna(0)
    
    train_set_for_vendor['target'] = (train_set_for_vendor['akeed_order_id_count']>0)*1
    
    # make test subset for vendor_id with target
    test_set_for_vendor = ss.loc[ss['VENDOR'] == vendor_id]

    test_set_for_vendor.columns = ['CID X LOC_NUM X VENDOR', 'target', 'customer_id',
                                   'location_number', 'vendor_id']
    
    
    # add customer features
    train_set_for_vendor_with_customer_features = pd.merge(train_set_for_vendor,
                                                       train_customers,how = 'left',
                                                       left_on ='customer_id',
                                                       right_on = 'akeed_customer_id' )
    
    test_set_for_vendor_with_customer_features = pd.merge(test_set_for_vendor,
                                                       test_customers,how = 'left',
                                                       left_on ='customer_id',
                                                       right_on = 'akeed_customer_id' )
    
    test_set_for_vendor_with_customer_features['location_number'] = test_set_for_vendor_with_customer_features['location_number'].astype(int)
    
    
    #add locations
    test_set_for_vendor_with_customer_features = pd.merge(test_set_for_vendor_with_customer_features,test_locations,
                                                     how = 'left', on = ['customer_id', 'location_number']
                                                     )
    
    
    #process features
    latitude_for_vendor_id, longitude_for_vendor_id = get_lat_long_for_vendor(vendor_id)
    
    
    
    train_set_for_vendor_with_customer_features['dist'] = dist_from_customer_to_vendor( train_set_for_vendor_with_customer_features['latitude'],
                             train_set_for_vendor_with_customer_features['longitude'],
                            latitude_for_vendor_id, longitude_for_vendor_id 
                            )
    
    
    test_set_for_vendor_with_customer_features['dist'] = dist_from_customer_to_vendor(test_set_for_vendor_with_customer_features['latitude'],
                             test_set_for_vendor_with_customer_features['longitude'],
                            latitude_for_vendor_id, longitude_for_vendor_id 
                            )
    
    
    train_set_for_vendor_with_customer_features ['language'] = \
        language_process(train_set_for_vendor_with_customer_features ['language'])
    
    
    test_set_for_vendor_with_customer_features ['language'] = \
        language_process(test_set_for_vendor_with_customer_features ['language'])
    
    
    
    train_set_for_vendor_with_customer_features ['gender'] = \
        gender_process(train_set_for_vendor_with_customer_features ['gender'])

    test_set_for_vendor_with_customer_features ['gender'] = \
        gender_process(test_set_for_vendor_with_customer_features ['gender'])
    
    
    train_set_for_vendor_with_customer_features ['dob'] = \
        dob_process(train_set_for_vendor_with_customer_features ['dob'])

    test_set_for_vendor_with_customer_features ['dob'] = \
        dob_process(test_set_for_vendor_with_customer_features ['dob'])
    
    
    features = ['gender', 'dob', 'status', 'verified' , 'language', 'dist']
    
   

    kf = KFold(n_splits=5)

    roc_auc_test_array = []

    best_f1_score_array, best_threshold_array = [], []

    
    
    for train_index, test_index in kf.split(train_set_for_vendor_with_customer_features[features]):
        X_train, X_test = train_set_for_vendor_with_customer_features.loc[train_index, features],\
                train_set_for_vendor_with_customer_features.loc[test_index, features]
        y_train, y_test = train_set_for_vendor_with_customer_features.loc[train_index, 'target'],\
                    train_set_for_vendor_with_customer_features.loc[test_index, 'target']

        model = LGBMClassifier(n_estimators=300, max_depth=5)
        model.fit(X_train, y_train)
        test_prediction = model.predict_proba(X_test)[:,1]

        #f1_score_test =f1_score(y_true= y_test, y_pred= test_prediction)
        roc_auc_test =roc_auc_score(y_true= y_test, y_score= test_prediction)
        roc_auc_test_array.append(roc_auc_test)
        #print(f1_score_test, roc_auc_test )
        print(roc_auc_test )


        best_f1_score, best_threshold = select_best_f1(y_true= y_test, y_score= test_prediction)
        print(best_f1_score, best_threshold)

        best_f1_score_array.append(best_f1_score)
        best_threshold_array.append(best_threshold)

    # todo    комментарии
    print( np.mean(roc_auc_test_array))
    
    
    #['gender', 'dob', 'status', 'verified' , 'language', 'dist']
    print(model.feature_importances_)
    median_best_threshold_array = np.median(best_threshold_array)
    
    median_best_threshold_array
    
    
    
    # final test
    X =train_set_for_vendor_with_customer_features[features]
    y = train_set_for_vendor_with_customer_features['target']
    model.fit(X,y)

    X_test =test_set_for_vendor_with_customer_features[features]
    final_predict = (model.predict_proba(X_test)[:,1] >median_best_threshold_array)*1

    return final_predict
    #ss.loc[ss['VENDOR']==vendor_id, 'target']


In [40]:
final_predict = make_solution_for_vendor(105)

Make prediction for vendor 105
0.7641903276359491


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.227111426543648 0.14141414141414144
0.7609847486900266


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2149220489977728 0.11111111111111112
0.76744785484043


...

0.2303899596593456 0.10101010101010102
0.7485929943657346


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.20743034055727552 0.09090909090909091
0.7839591005108406


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.21924144310823312 0.10101010101010102
0.7650350052085961
[ 225 1434  113  223  531 3604]


In [41]:
L = vendors['id'].nunique()
for i, vendor_id in enumerate(vendors['id'].unique()):
    try:
        print("process {} of {}".format(str(i+1), str(L)))
        predict_for_vendor =  make_solution_for_vendor(vendor_id)
        ss.loc[ss['VENDOR']==vendor_id, 'target'] = final_predict
    except:
        print("error for vendor_id: {}".format(str(vendor_id)))

process 1 of 100
Make prediction for vendor 4
0.8257521988192642


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.19507748404740202 0.08080808080808081
0.8437360395161694


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.1798201798201798 0.09090909090909091
0.8532366952542916


...

0 0
0.4803364357362559


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3636363636363636 0.21212121212121213
0.5029679492578506


C:\p\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.015873015873015872 0.010101010101010102
0.5948713031411863
[ 79 429  27 138 190 763]


In [42]:
ss.to_csv('submit.csv', index= None)